In [1]:
# Project setup
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

## LLM Graph Transformer

In [2]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4.1", temperature=0, openai_api_key=api_key)

# Initialize the graph transformer
graph_transformer = LLMGraphTransformer(llm=llm)

In [3]:
# Load the information file that we want to process
with open("elon_musk_profile.txt", "r") as file:
    content = file.read()

if content:
    print("File read successfully")

File read successfully


In [4]:
# Create the list of graph documents
documents = [Document(page_content=content)]
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [5]:
# Check the graph document
print(f"Nodes: {graph_documents[0].nodes}")
print(f"Relationships: {graph_documents[0].relationships}")

Nodes: [Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Maye Musk', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='Pretoria, South Africa', type='Place', properties={}), Node(id='June 28, 1971', type='Date', properties={}), Node(id="Queen'S University", type='Organization', properties={}), Node(id='University Of Pennsylvania', type='Organization', properties={}), Node(id='Stanford University', type='Organization', properties={}), Node(id='Tesla Inc.', type='Organization', properties={}), Node(id='Spacex', type='Organization', properties={}), Node(id='X', type='Organization', properties={}), Node(id='Neuralink', type='Organization', properties={}), Node(id='The Boring Company', type='Organization', properties={}), Node(id='Xai', type='Organization', properties={}), Node(id='Openai', type='Organization', properties={}), Node(id='Starlink', type='Organization', properties={}), Node(id='Falcon 9', type='Product', properties={

## Visualize Graph

In [6]:
from pyvis.network import Network
from pathlib import Path

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                  notebook=False, bgcolor="#222222", font_color="white")
    
    # Define the nodes and relationships
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)

    # Save the graph
    current_dir = Path.cwd()
    output_path = current_dir / "knowledge_graph.html"
    net.save_graph(str(output_path))
    print(f"Graph saved to {output_path}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{output_path}")
    except:
        print("Could not open browser automatically")
        
# Run the function
visualize_graph(graph_documents)

Graph saved to e:\NLP Learning\NLP-Learning\Knowledge Graph Visualizer\knowledge_graph.html


### Extract Specific Types of Nodes

In [13]:
# Print distinct node types
node_types = set(node.type for node in graph_documents[0].nodes)
print(f"Distinct Node Types: {node_types}")

Distinct Node Types: {'Place', 'Concept', 'Product', 'Award', 'Person', 'Value', 'Date', 'Organization'}


In [ ]:
# Select the allowed nodes
allowed_nodes = ["Person", "Organization", "Award", "ResearchField"]

# Initialize the graph transformer with the allowed nodes
graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes)

# Convert the documents to graph documents with the allowed nodes
graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)

### Extract Specific Types of Relationship

In [14]:
# Select the allowed relationships
allowed_nodes = ["Person", "Organization", "Award", "ResearchField"]

allowed_relationships = [
    ("Person", "WORKS_AT", "Organization"),
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]

# Initialize the graph transformer with the allowed relationships
graph_transformer_rel_defined = LLMGraphTransformer(
  llm=llm,
  allowed_nodes=allowed_nodes,
  allowed_relationships=allowed_relationships
)

# Convert the documents to graph documents with the allowed relationships
graph_documents_rel_defined = await graph_transformer_rel_defined.aconvert_to_graph_documents(documents)

In [15]:
# Visualize graph
visualize_graph(graph_documents_rel_defined)

Graph saved to e:\NLP Learning\NLP-Learning\Knowledge Graph Visualizer\knowledge_graph.html
